In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
titanic_train = pd.read_csv('train.csv')
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
885,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
886,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,45.0,1,2,W./C. 6607,23.4500,NaN,S
887,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.5+ KB


In [4]:
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [5]:
label_encoder = preprocessing.LabelEncoder()
titanic_train['Sex'] = label_encoder.fit_transform(titanic_train['Sex'])
titanic_train['Embarked'] = label_encoder.fit_transform(titanic_train['Embarked'])

In [6]:
cols = [ 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked']

In [7]:
rf_model = RandomForestClassifier(n_estimators = 1000, max_features = 2, oob_score = True)

In [8]:
rf_model.fit(titanic_train[cols], titanic_train['Survived'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
print('OOB Accuracy: ',rf_model.oob_score_)

OOB Accuracy:  0.8087739032620922


In [10]:
for cols, imp in zip(cols, rf_model.feature_importances_):
    print(cols, imp)

Pclass 0.08576335699005062
Sex 0.2613478389226062
Age 0.25513221182723617
SibSp 0.050054528238452876
Parch 0.03997235611275487
Fare 0.2726314560026807
Embarked 0.035098251906218514


In [11]:
features = ['Sex','Age','Fare']

In [12]:
dtree = tree.DecisionTreeClassifier()
dtree.fit(titanic_train[features], titanic_train['Survived'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [13]:
with open('Dtree1.dot','w') as f:
    f = tree.export_graphviz(dtree, feature_names = ['Sex','Age','Fare'], out_file=f)

In [14]:
predictors = pd.DataFrame(titanic_train[features])
predictors

,Sex,Age,Fare
0,1,22.0,7.2500
1,0,38.0,71.2833
2,0,26.0,7.9250
3,0,35.0,53.1000
4,1,35.0,8.0500
...,...,...,...
884,1,27.0,13.0000
885,0,19.0,30.0000
886,0,45.0,23.4500
887,1,26.0,30.0000


In [15]:
dtree.score(predictors, titanic_train['Survived'])

0.9763779527559056

In [16]:
titanic_test = pd.read_csv('test.csv')
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [17]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  417 non-null    int64  
 1   Pclass       417 non-null    int64  
 2   Name         417 non-null    object 
 3   Sex          417 non-null    object 
 4   Age          417 non-null    float64
 5   SibSp        417 non-null    int64  
 6   Parch        417 non-null    int64  
 7   Ticket       417 non-null    object 
 8   Fare         417 non-null    float64
 9   Embarked     417 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 32.7+ KB


In [18]:
titanic_test['Sex'] = label_encoder.fit_transform(titanic_test['Sex'])
titanic_test['Embarked'] = label_encoder.fit_transform(titanic_test['Embarked'])

In [19]:
test_features = pd.DataFrame(titanic_test[features])

In [20]:
test_preds = dtree.predict(test_features)

In [21]:
predicted_output = pd.DataFrame({ 'PassengerID' : titanic_test['PassengerId'],
                                'Survived' : test_preds})

In [22]:
predicted_output.to_csv("Output.csv", index = False)

In [23]:
predicted_output

,PassengerID,Survived
0,892,0
1,893,1
2,894,0
3,895,1
4,896,0
...,...,...
412,1305,0
413,1306,1
414,1307,0
415,1308,0
